### This script contains the following sections:

#### 1. Importing libraries and data 
#### 2. Integrity checks
#### 3. Addtional Data Joins
#### 4. Additional Data Cleaning

# 1. Imports

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\data_emergency_ex6.1\Data'

In [3]:
#import datasets to join

df_main = pd.read_csv(os.path.join(path,'Prepared','2.miss_val_check.csv'))

df_main.head()

,datayear,oshpd_id2,county_name,dba_address1,dba_city,dba_zip_code,license_category_desc,er_service_level_desc,teaching_hospital_desc,licensed_bed_size,...,sn_ic_care,medi_cal,medicare,other_payer,private_coverage,selfpay,ed_visit,all_other,english,spanish
0,2005,106010735,Alameda,2070 CLINTON,ALAMEDA,94501,General Acute Care,Basic,0.0,100-149,...,152.0,1285.0,2121.0,2624.0,5718.0,NaN,14473.0,NaN,NaN,NaN
1,2005,106010739,Alameda,2450 ASHBY STREET,BERKELEY,94705,General Acute Care,Basic,0.0,300-499,...,145.0,7226.0,6548.0,1974.0,13737.0,NaN,35018.0,NaN,NaN,NaN
2,2005,106010776,Alameda,747 52ND STREET,OAKLAND,94609,General Acute Care,Basic,0.0,150-199,...,NaN,29553.0,27.0,2303.0,9382.0,NaN,44788.0,NaN,NaN,NaN
3,2005,106010805,Alameda,20103 LAKE CHABOT ROAD,CASTRO VALLEY,94546,General Acute Care,Basic,0.0,150-199,...,1.0,4924.0,4171.0,774.0,10114.0,NaN,24278.0,NaN,NaN,NaN
4,2005,106010846,Alameda,1411 EAST 31ST STREET,OAKLAND,94602,General Acute Care,Basic,1.0,200-299,...,34.0,10925.0,2722.0,23181.0,2257.0,NaN,50083.0,NaN,NaN,NaN


In [4]:
# df_main['licensed_bed_size'].value_counts(dropna = False)

In [5]:
df_add = pd.read_csv(os.path.join(path,'Original','1.Raw','encounters-by-facility.csv'))

df_add.head()

,year,oshpd_id,facility_name,county_name,er_service_level_desc,type,count
0,2012,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2595.0
1,2012,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Visit,13727.0
2,2013,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2579.0
3,2013,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Visit,13538.0
4,2014,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2214.0


In [6]:
print('Main df'+ str(df_main.shape), 'supple_df' + str(df_add.shape))

Main df(5199, 50) supple_df(5922, 7)


# 2. Integrity Checks

In [7]:
# check the number of unique ids in each dataset

print(df_main.groupby(['oshpd_id2'])['oshpd_id2'].unique().count(),
      df_add.groupby(['oshpd_id'])['oshpd_id'].unique().count())

361 368


In [8]:
# Check for na in addtional dataset
df_add.isna().sum()

year                       0
oshpd_id                   0
facility_name              0
county_name                0
er_service_level_desc    132
type                       0
count                     77
dtype: int64

In [9]:
df_add['er_service_level_desc'] = df_add['er_service_level_desc'].fillna('UNKNOWN')

# df_add['er_service_level_desc'] = df_add['er_service_level_desc'].str.lower()

df_add.isna().sum()

year                      0
oshpd_id                  0
facility_name             0
county_name               0
er_service_level_desc     0
type                      0
count                    77
dtype: int64

# 3. Additional Data Joins

In [10]:
#since the supplementary dataset has shorter time periods, the supple dataset will become the main dataset.

df_combined = df_add.merge(df_main, left_on = ['oshpd_id','year'], right_on = ['oshpd_id2','datayear'],
             how = 'inner', indicator=True)
df_combined['_merge'].value_counts(dropna = False)

both          5782
right_only       0
left_only        0
Name: _merge, dtype: int64

In [11]:
comp = np.where(df_combined['er_service_level_desc_x'].str.lower() == df_combined['er_service_level_desc_y'].str.lower(), 
                True, False)
df_combined['same'] = comp

#Inconsistencies found in the the two datasets. Mainly, varying levels assigned to the same hospitals in the same year
#E.g. Basic and Standby

In [12]:
#Drop columns that are not relevant or redundant the dataset

df_combined.drop(columns = ['oshpd_id2','county_name_y','er_service_level_desc_y','_merge','same','ed_visit','datayear'],
                 inplace = True)

In [13]:
df_combined.rename(columns = {'county_name_x': 'county_name',
                              'type': 'encounter_type',
                              'count': 'encounter_type_total',
                              'er_service_level_desc_x': 'er_service_level'}, inplace = True)

In [14]:
df_combined.columns

Index(['year', 'oshpd_id', 'facility_name', 'county_name', 'er_service_level',
       'encounter_type', 'encounter_type_total', 'dba_address1', 'dba_city',
       'dba_zip_code', 'license_category_desc', 'teaching_hospital_desc',
       'licensed_bed_size', 'mssa_designation', 'sex_male', 'sex_female',
       'sex_unk', 'age_01_09', 'age_10_19', 'age_20_29', 'age_30_39',
       'age_40_49', 'age_50_59', 'age_60_69', 'age_70_79', 'age_80_',
       'age_under_1', 'age_unknown', 'asian_pacific_island', 'black',
       'hispanic', 'native_american_eski', 'other_race', 'unknown_race',
       'white', 'acute_care', 'against_medical_advice', 'died', 'hospice_care',
       'home_health_service', 'residential_care', 'routine',
       'psychiatric_care', 'sn_ic_care', 'medi_cal', 'medicare', 'other_payer',
       'private_coverage', 'selfpay', 'all_other', 'english', 'spanish'],
      dtype='object')

In [15]:
df_codes = pd.read_csv((os.path.join(path,'Original','1.Raw','county_codes.csv')))
df_codes.shape

(58, 2)

In [16]:
df_codes.head()

,county_number,county_name
0,1,Alameda
1,2,Alpine
2,3,Amador
3,4,Butte
4,5,Calaveras


In [17]:
df_codes['county_name'] = df_codes['county_name'].str.upper()

In [18]:
# add county codes to the list
df_combined_county = df_combined.merge(df_codes, on = 'county_name', how = 'left', indicator = True)
df_combined_county.head()

,year,oshpd_id,facility_name,county_name,er_service_level,encounter_type,encounter_type_total,dba_address1,dba_city,dba_zip_code,...,medi_cal,medicare,other_payer,private_coverage,selfpay,all_other,english,spanish,county_number,_merge
0,2012,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2595.0,2070 CLINTON,ALAMEDA,94501,...,549.0,2740.0,3469.0,4339.0,NaN,522.0,12918.0,287.0,1,both
1,2012,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Visit,13727.0,2070 CLINTON,ALAMEDA,94501,...,549.0,2740.0,3469.0,4339.0,NaN,522.0,12918.0,287.0,1,both
2,2013,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2579.0,2070 CLINTON AVE,ALAMEDA,94501,...,373.0,2788.0,3493.0,4306.0,NaN,600.0,12671.0,267.0,1,both
3,2013,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Visit,13538.0,2070 CLINTON AVE,ALAMEDA,94501,...,373.0,2788.0,3493.0,4306.0,NaN,600.0,12671.0,267.0,1,both
4,2014,106010735,ALAMEDA HOSPITAL,ALAMEDA,BASIC,ED_Admit,2214.0,2070 CLINTON,ALAMEDA,94501,...,732.0,2769.0,4282.0,3854.0,2390.0,551.0,13176.0,300.0,1,both


In [19]:
df_combined_county['_merge'].value_counts(dropna = False)

both          5782
right_only       0
left_only        0
Name: _merge, dtype: int64

In [20]:
print(df_combined_county.columns.tolist())

['year', 'oshpd_id', 'facility_name', 'county_name', 'er_service_level', 'encounter_type', 'encounter_type_total', 'dba_address1', 'dba_city', 'dba_zip_code', 'license_category_desc', 'teaching_hospital_desc', 'licensed_bed_size', 'mssa_designation', 'sex_male', 'sex_female', 'sex_unk', 'age_01_09', 'age_10_19', 'age_20_29', 'age_30_39', 'age_40_49', 'age_50_59', 'age_60_69', 'age_70_79', 'age_80_', 'age_under_1', 'age_unknown', 'asian_pacific_island', 'black', 'hispanic', 'native_american_eski', 'other_race', 'unknown_race', 'white', 'acute_care', 'against_medical_advice', 'died', 'hospice_care', 'home_health_service', 'residential_care', 'routine', 'psychiatric_care', 'sn_ic_care', 'medi_cal', 'medicare', 'other_payer', 'private_coverage', 'selfpay', 'all_other', 'english', 'spanish', 'county_number', '_merge']


In [21]:
# Change order of columns and also drop "_merge" and "datayear" column 

df_combined_county = df_combined_county[['year','oshpd_id','county_number', 'county_name', 'facility_name',
                             'dba_address1', 'dba_city', 'dba_zip_code', 
                             'license_category_desc', 'teaching_hospital_desc', 'licensed_bed_size',
                             'mssa_designation','er_service_level', 'encounter_type', 'encounter_type_total', 'sex_male', 'sex_female', 'sex_unk', 'age_01_09', 'age_10_19', 
                             'age_20_29', 'age_30_39', 'age_40_49', 'age_50_59', 'age_60_69', 'age_70_79', 'age_80_',
                             'age_under_1', 'age_unknown', 'asian_pacific_island', 'black', 'hispanic',
                             'native_american_eski', 'other_race', 'unknown_race', 'white', 'acute_care', 
                             'against_medical_advice', 'died', 'hospice_care', 'home_health_service', 'residential_care', 
                             'routine', 'psychiatric_care', 'sn_ic_care', 'medi_cal', 'medicare', 'other_payer',
                             'private_coverage', 'selfpay', 'all_other', 'english', 'spanish']]

# 4. Data Cleaning

In [22]:
# Impute numerical columns with 0

df_combined_county.loc[:,'sex_unk':'spanish'] = df_combined_county.loc[:,'sex_unk':'spanish'].fillna(0)

In [23]:
df_combined_county.head()

,year,oshpd_id,county_number,county_name,facility_name,dba_address1,dba_city,dba_zip_code,license_category_desc,teaching_hospital_desc,...,psychiatric_care,sn_ic_care,medi_cal,medicare,other_payer,private_coverage,selfpay,all_other,english,spanish
0,2012,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON,ALAMEDA,94501,General Acute Care,0.0,...,210.0,201.0,549.0,2740.0,3469.0,4339.0,0.0,522.0,12918.0,287.0
1,2012,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON,ALAMEDA,94501,General Acute Care,0.0,...,210.0,201.0,549.0,2740.0,3469.0,4339.0,0.0,522.0,12918.0,287.0
2,2013,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON AVE,ALAMEDA,94501,General Acute Care,0.0,...,198.0,220.0,373.0,2788.0,3493.0,4306.0,0.0,600.0,12671.0,267.0
3,2013,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON AVE,ALAMEDA,94501,General Acute Care,0.0,...,198.0,220.0,373.0,2788.0,3493.0,4306.0,0.0,600.0,12671.0,267.0
4,2014,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON,ALAMEDA,94501,General Acute Care,0.0,...,179.0,179.0,732.0,2769.0,4282.0,3854.0,2390.0,551.0,13176.0,300.0


In [24]:
#seperate columns from the target pivot columns

piv_cols = [col for col in df_combined_county.columns if col not in ['encounter_type', 'encounter_type_total']]
# piv_cols

In [25]:
# Pivot 'er_service_level', 'encounter_type' columns to make two columns for the former with values from the latter

# year 2012 was removed when pivot_table was used. 
df_combined_pivoted = df_combined_county.pivot(index = piv_cols,
                                columns ='encounter_type', values='encounter_type_total').reset_index()
# df_combined_pivoted.columns.name = None

df_combined_pivoted.columns = df_combined_pivoted.columns.str.lower()

df_combined_pivoted

encounter_type,year,oshpd_id,county_number,county_name,facility_name,dba_address1,dba_city,dba_zip_code,license_category_desc,teaching_hospital_desc,...,medi_cal,medicare,other_payer,private_coverage,selfpay,all_other,english,spanish,ed_admit,ed_visit
0,2012,106010735,1,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON,ALAMEDA,94501,General Acute Care,0.0,...,549.0,2740.0,3469.0,4339.0,0.0,522.0,12918.0,287.0,2595.0,13727.0
1,2012,106010739,1,ALAMEDA,ALTA BATES SUMMIT MEDICAL CENTER-ALTA BATES CA...,2450 ASHBY STREET,BERKELEY,94705,General Acute Care,0.0,...,8613.0,5707.0,1877.0,12926.0,0.0,534.0,33867.0,941.0,6047.0,35342.0
2,2012,106010776,1,ALAMEDA,CHILDRENS HOSPITAL AND RESEARCH CENTER AT OAKLAND,747 52ND STREET,OAKLAND,94609,General Acute Care,0.0,...,28489.0,19.0,1785.0,5949.0,0.0,1425.0,27285.0,10508.0,6911.0,39218.0
3,2012,106010805,1,ALAMEDA,EDEN MEDICAL CENTER,20103 LAKE CHABOT ROAD,CASTRO VALLEY,94546,General Acute Care,0.0,...,8543.0,6426.0,814.0,9169.0,0.0,666.0,29433.0,1632.0,6324.0,31731.0
4,2012,106010846,1,ALAMEDA,HIGHLAND HOSPITAL,1411 EAST 31ST STREET,OAKLAND,94602,General Acute Care,1.0,...,17877.0,4437.0,35836.0,1640.0,0.0,4025.0,57871.0,12306.0,8992.0,74202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020,106560525,56,VENTURA,ADVENTIST HEALTH SIMI VALLEY,2975 Sycamore Dr,Simi Valley,93065,General Acute Care,0.0,...,8941.0,5341.0,641.0,8516.0,1584.0,116.0,23065.0,1866.0,5709.0,25051.0
2887,2020,106560529,56,VENTURA,ST. JOHN'S REGIONAL MEDICAL CENTER,1600 N Rose Ave,Oxnard,93030,General Acute Care,0.0,...,18418.0,6674.0,1863.0,5037.0,3299.0,143.0,26822.0,8255.0,6578.0,35291.0
2888,2020,106571086,57,YOLO,WOODLAND MEMORIAL HOSPITAL,1325 Cottonwood St,Woodland,95695,General Acute Care,0.0,...,8618.0,4075.0,447.0,4387.0,1397.0,399.0,15962.0,2537.0,2206.0,18924.0
2889,2020,106574010,57,YOLO,SUTTER DAVIS HOSPITAL,2000 Sutter Pl,Davis,95616,General Acute Care,0.0,...,7814.0,4540.0,665.0,6935.0,1177.0,609.0,18037.0,2443.0,1986.0,21131.0


In [26]:
df_combined_pivoted.shape

(2891, 53)

In [27]:
# Check for duplicate values

df_combined_pivoted[df_combined_pivoted.duplicated()]

encounter_type,year,oshpd_id,county_number,county_name,facility_name,dba_address1,dba_city,dba_zip_code,license_category_desc,teaching_hospital_desc,...,medi_cal,medicare,other_payer,private_coverage,selfpay,all_other,english,spanish,ed_admit,ed_visit


In [28]:
df_combined_pivoted.dropna(axis = 0, how = 'any', subset = ['ed_admit'], inplace = True)
df_combined_pivoted.drop(columns = ['mssa_designation'], inplace = True)

In [29]:
df_combined_pivoted.isna().sum()

encounter_type
year                      0
oshpd_id                  0
county_number             0
county_name               0
facility_name             0
dba_address1              0
dba_city                  0
dba_zip_code              0
license_category_desc     0
teaching_hospital_desc    0
licensed_bed_size         0
er_service_level          0
sex_male                  0
sex_female                0
sex_unk                   0
age_01_09                 0
age_10_19                 0
age_20_29                 0
age_30_39                 0
age_40_49                 0
age_50_59                 0
age_60_69                 0
age_70_79                 0
age_80_                   0
age_under_1               0
age_unknown               0
asian_pacific_island      0
black                     0
hispanic                  0
native_american_eski      0
other_race                0
unknown_race              0
white                     0
acute_care                0
against_medical_advice    0
died 

In [30]:
df_combined_pivoted['county_name'] = df_combined_pivoted['county_name'].str.title()

In [31]:
df_combined_pivoted['county_name'].unique()

array(['Alameda', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura',
       'Yolo', 'Yuba'], dtype=object)

In [32]:
df_combined_pivoted['dba_zip_code'] = df_combined_pivoted['dba_zip_code'].astype('str')

In [33]:
df_combined_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2883 entries, 0 to 2890
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    2883 non-null   int64  
 1   oshpd_id                2883 non-null   int64  
 2   county_number           2883 non-null   int64  
 3   county_name             2883 non-null   object 
 4   facility_name           2883 non-null   object 
 5   dba_address1            2883 non-null   object 
 6   dba_city                2883 non-null   object 
 7   dba_zip_code            2883 non-null   object 
 8   license_category_desc   2883 non-null   object 
 9   teaching_hospital_desc  2883 non-null   float64
 10  licensed_bed_size       2883 non-null   object 
 11  er_service_level        2883 non-null   object 
 12  sex_male                2883 non-null   float64
 13  sex_female              2883 non-null   float64
 14  sex_unk                 2883 non-null   

In [34]:
# df_combined_pivoted.to_csv(os.path.join(path,'Prepared','3.data_join_facilities.csv'), index = 0)